In [57]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re

from IPython.display import display, Latex, Markdown
#from client.api.notebook import Notebook
#ok = Notebook('proj1.ok')

In [58]:
sent = pd.read_csv('data/vader_lexicon.txt', sep='\t', header=None)
sent = sent.drop([2, 3], 1)
sent.index.rename('token', inplace=True)
sent.rename(columns={1: 'polarity'}, inplace=True)
sent.head()
#print(len(sent))

,0,polarity
token,,
0,$:,-1.5
1,%),-0.4
2,%-),-1.5
3,&-:,-0.4
4,&:,-0.7


In [59]:
from datetime import datetime

ELEC_DATE = datetime(2016, 11, 8)
INAUG_DATE = datetime(2017, 1, 20)

In [60]:
trump_tweets = pd.read_csv('data/trump-tweet-in-office.csv')
trump_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,10-25-2019 02:10:02,8819,0,True,1187551862616141824
1,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,10-25-2019 02:08:26,13713,0,True,1187551461808492544
2,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,10-25-2019 02:08:24,10961,0,True,1187551451775750144
3,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,10-25-2019 02:07:49,4547,0,True,1187551305281212416
4,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,10-25-2019 02:07:37,10491,0,True,1187551256371515392


In [61]:
trump = trump_tweets.copy()
trump['id']=trump['id_str']
trump = trump.set_index(['id'])
trump.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
id,,,,,,,
1187551862616141824,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,10-25-2019 02:10:02,8819,0,True,1187551862616141824
1187551461808492544,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,10-25-2019 02:08:26,13713,0,True,1187551461808492544
1187551451775750144,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,10-25-2019 02:08:24,10961,0,True,1187551451775750144
1187551305281212416,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,10-25-2019 02:07:49,4547,0,True,1187551305281212416
1187551256371515392,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,10-25-2019 02:07:37,10491,0,True,1187551256371515392


In [62]:
repl_punc = trump['text'].str.replace(r"[?|,|$|!|&|.|#|^|;|:|/|\"]", ' ')
replacequotes = repl_punc.str.replace(r'^"', '')
#replacequotes = repl_punc.strip('"')
#replacequotes.loc['907675638055743489'].strip('\'"') 
#repl_punc.loc['1187551862616141824']

In [63]:
# Save your regex in punct_re
punct_re = r"[^\w\s]"
#[?|,|$|!|&|.|#|^|;|:|/|\"]
trump['no_punc'] = trump['text'].str.replace(punct_re, " ")

In [64]:
trumpcopy = trump.copy()
changed_punc = trumpcopy['no_punc'].str.replace('\s+', ' ')
trumpcopy['no_punc'] = changed_punc
splt = trumpcopy['no_punc'].str.split(expand=True).stack().reset_index().set_index('id')
splt.columns = ['num', 'word']


In [65]:
splt.head(10)


,num,word
id,,
1187551862616141824,0,RT
1187551862616141824,1,RepLeeZeldin
1187551862616141824,2,This
1187551862616141824,3,impeachment
1187551862616141824,4,inquiry
1187551862616141824,5,has
1187551862616141824,6,produced
1187551862616141824,7,NOTHING
1187551862616141824,8,to


In [66]:
tidy_format = splt
newt = tidy_format.reset_index()
newsent = sent.reset_index().rename(columns = {'token': 'word'})

In [67]:
display(newt.head())
display(newsent.head())

,id,num,word
0,1187551862616141824,0,RT
1,1187551862616141824,1,RepLeeZeldin
2,1187551862616141824,2,This
3,1187551862616141824,3,impeachment
4,1187551862616141824,4,inquiry


,word,0,polarity
0,0,$:,-1.5
1,1,%),-0.4
2,2,%-),-1.5
3,3,&-:,-0.4
4,4,&:,-0.7


In [68]:
polar = pd.merge(newt, newsent, how='outer', on= 'word')
newpol = polar.groupby('id').sum()
newpol = newpol.drop('num', 1).fillna(0)
display(newpol.head())
display(newsent.head())

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat